In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 迁移：自动ML图像分类

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-image-classification-batch-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-image-classification-batch-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-image-classification-batch-online.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署AutoML图像分类模型。

了解更多关于[迁移至Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[图像数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_images)的信息。

### 目标

在本教程中，您将学习如何使用 `AutoML` 训练图像模型，并使用 `Vertex AI Prediction` 和 `Vertex AI batch prediction` 进行在线和批量预测。

本教程使用以下 Google Cloud 机器学习服务：

- `AutoML`
- `Vertex AI batch prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 训练一个 AutoML 图像分类模型。
- 进行批量预测。
- 将模型部署到一个端点。
- 进行在线预测。

数据集

本教程使用的数据集为来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练后的模型可以预测图像所属的花卉类型，包括：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow

只有 Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在您开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用一个共享的测试账户或项目。为了避免在创建资源时出现用户之间的名称冲突，您为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关指示操作。

1. 顶点 AI 工作台
* 不要做任何操作，因为您已经通过验证。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务账号授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库和定义常量

In [ ]:
import os

import google.cloud.aiplatform as aip

## 为Python初始化Vertex AI SDK

为您的项目和相应的存储桶初始化Vertex AI SDK for Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的Flowers数据集的一个版本，使用CSV索引文件。

首先，快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

## 创建一个数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Flowers" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

示例输出：

INFO：google.cloud.aiplatform.datasets.dataset：创建图像数据集
INFO：google.cloud.aiplatform.datasets.dataset：创建图像数据集支持的LRO：projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/1941426647739662336
INFO：google.cloud.aiplatform.datasets.dataset：创建了图像数据集。资源名称：projects/759209241365/locations/us-central1/datasets/2940964905882222592
INFO：google.cloud.aiplatform.datasets.dataset：要在另一个会话中使用此图像数据集：
INFO：google.cloud.aiplatform.datasets.dataset：ds = aiplatform.ImageDataset（'projects/759209241365/locations/us-central1/datasets/2940964905882222592'）
INFO：google.cloud.aiplatform.datasets.dataset：正在导入ImageDataset数据：projects/759209241365/locations/us-central1/datasets/2940964905882222592
INFO：google.cloud.aiplatform.datasets.dataset：导入ImageDataset数据支持的LRO：projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/8100099138168815616
INFO：google.cloud.aiplatform.datasets.dataset：导入了图像数据集数据。资源名称：projects/759209241365/locations/us-central1/datasets/2940964905882222592
projects/759209241365/locations/us-central1/datasets/2940964905882222592

训练一个模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)
### [训练.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建并运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流水线，2) 运行该流水线。

#### 创建训练流水线

使用`AutoMLImageTrainingJob`类创建一个AutoML训练流水线，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：为模型训练指定任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则为单标签（`False`）或多标签（`True`）。
- `model_type`：部署模型的类型。
  - `CLOUD`：部署在谷歌云上。
  - `CLOUD_HIGH_ACCURACY_1`：针对谷歌云部署的准确性优先型模型。
  - `CLOUD_LOW_LATENCY_`：针对谷歌云部署的延迟优先型模型。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对边缘设备部署的准确性优先型模型。
  - `MOBILE_TF_LOW_LATENCY_1`：针对边缘设备部署的延迟优先型模型。
- `base_model`：（可选）从现有`Model`资源进行迁移学习 -- 仅支持图像分类。

实例化的对象是训练作业的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="flowers_" + UUID,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLImageTrainingJob object at 0x7f806a6116d0> 

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLImageTrainingJob object at 0x7f806a6116d0>

#### 运行训练流水线

接下来，您可以通过调用方法`run`来运行DAG，启动训练作业，参数如下：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 经过训练的模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`: 如果为`True`，当服务认为无法进一步改进模型的客观测量时，训练可能会在使用完整预算之前完成。

完成`run`方法后，返回`Model`资源。

训练流水线的执行最多需要20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

示例输出：

    信息：google.cloud.aiplatform.training_jobs:查看培训：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/2109316300865011712?project=759209241365
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    信息：google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob 运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712
    信息：google.cloud.aiplatform.training_jobs:模型可在项目/759209241365/locations/us-central1/models/1284590221056278528 找到。

## 评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [项目.位置.模型.评估.列表](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=flowers_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

示例输出：

    名称："projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    metrics_schema_uri："gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    metrics {
      struct_value {
        fields {
          key: "auPrc"
          value {
            number_value: 0.9891107
          }
        }
        fields {
          key: "confidenceMetrics"
          value {
            list_value {
              values {
                struct_value {
                  fields {
                    key: "precision"
                    value {
                      number_value: 0.2
                    }
                  }
                  fields {
                    key: "recall"
                    value {
                      number_value: 1.0
                    }
                  }
                }
              }

进行批量预测

[预测批处理](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

获取测试项目

现在对您的Vertex模型进行批量预测。您可以从数据集中任意选择示例作为测试项目。不必担心这些示例可能已经在训练模型时使用过 -- 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
if len(str(test_items[0]).split(",")) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项

对于批量预测，请将测试项复制到您的Cloud Storage存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

### 创建批处理输入文件

现在创建一个批处理输入文件，将其存储在您本地的云存储桶中。批处理输入文件可以是CSV或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您需要为每个数据项（实例）的每一行创建一个字典条目。该字典包含键/值对：

- `content`：图片的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个`jpeg`文件。

例如：

                        {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法发起批量预测，参数如下：

- `job_display_name`：批量预测任务的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，则调用将阻塞，等待异步批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="flowers_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

示例输出：

    INFO: google.cloud.aiplatform.jobs: 创建BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
    INFO: google.cloud.aiplatform.jobs: 创建了BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO: google.cloud.aiplatform.jobs: 要在另一个会话中使用此BatchPredictionJob：
    INFO: google.cloud.aiplatform.jobs: bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO: google.cloud.aiplatform.jobs: 查看Batch预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
    JobState.JOB_STATE_RUNNING

### 等待批处理预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    INFO: google.cloud.aiplatform.jobs: 创建了 BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO: google.cloud.aiplatform.jobs: 要在另一个会话中使用此 BatchPredictionJob：
    INFO: google.cloud.aiplatform.jobs: bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO: google.cloud.aiplatform.jobs: 查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    INFO: google.cloud.aiplatform.jobs: BatchPredictionJob 运行已完成。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### 获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果被写入到您在批量预测请求中指定的 Cloud Storage 输出存储桶中。您可以调用 `iter_outputs()` 方法来获取生成结果的每个 Cloud Storage 文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
 - `ids`: 每个预测请求的内部分配的唯一标识符。
 - `displayNames`: 每个类标签的类别名称。
 - `confidences`: 针对每个类标签的预测置信度，介于 0 和 1 之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*示例输出：*

{'instance': {'content': 'gs://andy-1234-221921aip-20210802180634/100080576_f52e8ee070_n.jpg', 'mimeType': 'image/jpeg'}, 'prediction': {'ids': ['3195476558944927744', '1636105187967893504', '7400712711002128384', '2789026692574740480', '5501319568158621696'], 'displayNames': ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'], 'confidences': [0.99998736, 8.222247e-06, 3.6782617e-06, 5.3231275e-07, 2.6960555e-07]}}

做在线预测

### [predictions.deploy-model-api] (https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [预测部署模型API] (https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

*示例输出：*

    信息：google.cloud.aiplatform.models:创建端点
    信息：google.cloud.aiplatform.models:创建端点支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    信息：google.cloud.aiplatform.models:端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:在另一个会话中使用此端点：
    信息：google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    信息：google.cloud.aiplatform.models:将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:部署端点模型支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    信息：google.cloud.aiplatform.models:端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)
### [在线预测自动ML](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

获取测试项

您可以从数据集中任意选择一个示例作为测试项。不必担心该示例可能已在训练模型时使用过 - 我们只是想演示如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源，您可以通过向Endpoint资源发送预测请求进行在线预测。

#### 请求

由于在本示例中，您的测试项目位于 Cloud Storage 桶中，您可以使用`tf.io.gfile.Gfile()`来打开和读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为 base64 -- 这样可以确保内容在通过网络传输二进制数据时不会被修改。

每个实例的格式如下：

    { 'content': { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将单个测试项目发送为包含一个测试项目的列表。

#### 响应

从`predict()`调用的响应是一个Python字典，包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类别标签的类名。
- `confidences`: 每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`: 执行预测的部署的Model资源的 Vertex AI 标识符。

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

*Example output:*

预测(predictions=[{'ids': ['3195476558944927744', '5501319568158621696', '1636105187967893504', '2789026692574740480', '7400712711002128384'], 'displayNames': ['daisy', 'tulips', 'dandelion', 'sunflowers', 'roses'], 'confidences': [0.999987364, 2.69604527e-07, 8.2222e-06, 5.32310196e-07, 3.6782335e-06]}], deployed_model_id='5949545378826158080', explanations=None)

取消部署模型

当您完成预测时，您可以从“端点”资源中取消部署模型。这将取消所有计算资源，并停止为部署的模型计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您在教程中使用的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML trainig job
dag.delete()

# Delete the batch prediction job
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI